In [1]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)

if not dir1 in sys.path:
    sys.path.append(dir1)

os.chdir('..')

In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import yaml

from sklearn.metrics import mean_absolute_error as MAE
from matplotlib import pyplot as plt
from xgboost import XGBRegressor

from src.data.preprocessing import read_data, data_to_np_tensor, preprocess_split
from src.models.ts2vec_src.ts2vec import TS2Vec

%load_ext autoreload
%autoreload 2

/tmp/ipykernel_34890/235575361.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Preprocessing

In [3]:
# data = read_data('data/all_tickers.csv')
# train_data = preprocessing(
#     data, 
#     ['Open', 'High', 'Low', 'Close', 'Volume'],         
#     start_date = '2023-12-15',
#     end_date = '2023-12-21'
# )

# test_data = preprocessing(
#     data, 
#     ['Open', 'High', 'Low', 'Close', 'Volume'],          
#     start_date = '2023-12-21',
#     end_date = '2023-12-22',
#     tickers_save = train_data['Close'].columns
# )

In [4]:
# for feature in ['Open', 'High', 'Low', 'Close', 'Volume']:
#     print(train_data['Close'].shape[1], test_data['Close'].shape[1])

In [5]:
# train_ts = data_to_np_tensor(train_data)
# test_ts = data_to_np_tensor(test_data)

In [6]:
# model = TS2Vec(
#     input_dims=train_ts.shape[2],
#     device=1,
#     output_dims=320
# )

# loss_log = model.fit(
#     train_ts,
#     verbose=False
# )

# test_repr = model.encode(test_ts)
# #test_repr = model.encode(test_data, encoding_window='full_series')

In [7]:
# test_repr = model.encode(test_ts, encoding_window='full_series')
# print(test_repr.shape)
# test_repr

# mean stock price prediction

In [8]:
with open('configs/best_stocks_nans_rate.yaml') as f:
    best_stocks = yaml.load(f, Loader=yaml.FullLoader)
best_stocks = list(best_stocks.keys())
# best_stocks

In [9]:
df = read_data('data/all_tickers.csv')
df_best = df.query("Stock in @best_stocks")

In [10]:
df_best

,Date,Time,Open,High,Low,Close,Volume,Datetime,Stock,Day_week
61,2024-01-30,22:58:00,188.130,188.17,188.110,188.14,1500,2024-01-30 22:58:00,AAPL,Tuesday
62,2024-01-30,22:57:00,188.220,188.22,188.080,188.11,5700,2024-01-30 22:57:00,AAPL,Tuesday
63,2024-01-30,22:56:00,188.220,188.27,188.215,188.22,2600,2024-01-30 22:56:00,AAPL,Tuesday
64,2024-01-30,22:55:00,188.150,188.20,188.140,188.20,2100,2024-01-30 22:55:00,AAPL,Tuesday
65,2024-01-30,22:54:00,188.180,188.18,188.140,188.18,2013,2024-01-30 22:54:00,AAPL,Tuesday
...,...,...,...,...,...,...,...,...,...,...
95882,2023-01-30,17:34:00,115.050,115.09,114.960,114.97,800,2023-01-30 17:34:00,XOM,Monday
95883,2023-01-30,17:33:00,115.090,115.27,114.910,115.15,4603,2023-01-30 17:33:00,XOM,Monday
95884,2023-01-30,17:32:00,115.250,115.25,115.000,115.03,850,2023-01-30 17:32:00,XOM,Monday
95885,2023-01-30,17:31:00,115.145,115.16,115.145,115.16,237,2023-01-30 17:31:00,XOM,Monday


In [11]:
mask = df['Stock'].apply(lambda x: x in best_stocks) 

In [12]:
df[mask]

,Date,Time,Open,High,Low,Close,Volume,Datetime,Stock,Day_week
61,2024-01-30,22:58:00,188.130,188.17,188.110,188.14,1500,2024-01-30 22:58:00,AAPL,Tuesday
62,2024-01-30,22:57:00,188.220,188.22,188.080,188.11,5700,2024-01-30 22:57:00,AAPL,Tuesday
63,2024-01-30,22:56:00,188.220,188.27,188.215,188.22,2600,2024-01-30 22:56:00,AAPL,Tuesday
64,2024-01-30,22:55:00,188.150,188.20,188.140,188.20,2100,2024-01-30 22:55:00,AAPL,Tuesday
65,2024-01-30,22:54:00,188.180,188.18,188.140,188.18,2013,2024-01-30 22:54:00,AAPL,Tuesday
...,...,...,...,...,...,...,...,...,...,...
95882,2023-01-30,17:34:00,115.050,115.09,114.960,114.97,800,2023-01-30 17:34:00,XOM,Monday
95883,2023-01-30,17:33:00,115.090,115.27,114.910,115.15,4603,2023-01-30 17:33:00,XOM,Monday
95884,2023-01-30,17:32:00,115.250,115.25,115.000,115.03,850,2023-01-30 17:32:00,XOM,Monday
95885,2023-01-30,17:31:00,115.145,115.16,115.145,115.16,237,2023-01-30 17:31:00,XOM,Monday


In [13]:
dd = {'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}

In [14]:
df_best_h = df_best.set_index('Datetime').groupby(
    ['Stock', pd.Grouper(freq='h')],
).agg(dd)

df_best_h

Open    High     Low   Close  Volume
Stock Datetime                                                   
AAPL  2023-01-30 17:00:00  145.15  145.53  144.18  145.13  291457
      2023-01-30 18:00:00  143.97  145.33  143.38  144.96  348830
      2023-01-30 19:00:00  144.07  144.30  143.50  143.95  215404
      2023-01-30 20:00:00  143.38  144.24  143.18  144.17  254970
      2023-01-30 21:00:00  143.31  143.81  143.01  143.38  192383
...                           ...     ...     ...     ...     ...
XOM   2024-01-30 18:00:00  103.69  103.70  102.95  103.22  159389
      2024-01-30 19:00:00  103.79  103.96  103.49  103.67   90110
      2024-01-30 20:00:00  103.68  103.98  103.63  103.76   54009
      2024-01-30 21:00:00  104.15  104.16  103.67  103.68   61363
      2024-01-30 22:00:00  104.38  104.46  104.15  104.15   91004

[42196 rows x 5 columns]

In [15]:
df = df_best_h.groupby('Stock').pct_change().reset_index()

In [16]:
df

,Stock,Datetime,Open,High,Low,Close,Volume
0,AAPL,2023-01-30 17:00:00,NaN,NaN,NaN,NaN,NaN
1,AAPL,2023-01-30 18:00:00,-0.008130,-0.001374,-0.005549,-0.001171,0.196849
2,AAPL,2023-01-30 19:00:00,0.000695,-0.007087,0.000837,-0.006967,-0.382496
3,AAPL,2023-01-30 20:00:00,-0.004789,-0.000416,-0.002230,0.001528,0.183683
4,AAPL,2023-01-30 21:00:00,-0.000488,-0.002981,-0.001187,-0.005480,-0.245468
...,...,...,...,...,...,...,...
42191,XOM,2024-01-30 18:00:00,0.005430,0.003969,0.008325,0.008106,0.568944
42192,XOM,2024-01-30 19:00:00,0.000964,0.002507,0.005245,0.004360,-0.434654
42193,XOM,2024-01-30 20:00:00,-0.001060,0.000192,0.001353,0.000868,-0.400633
42194,XOM,2024-01-30 21:00:00,0.004533,0.001731,0.000386,-0.000771,0.136162


In [17]:
train_start, train_end = '2023-10-01', '2023-11-01'
test_start, test_end = '2023-11-01', '2023-11-07'


train_data = preprocess_split(
    df, 
    ['Open', 'High', 'Low', 'Close', 'Volume'],         
    start_date = train_start,
    end_date = train_end,
    tickers_save = best_stocks
)

test_data = preprocess_split(
    df, 
    ['Open', 'High', 'Low', 'Close', 'Volume'],          
    start_date = test_start,
    end_date = test_end,
    tickers_save = best_stocks
)

NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0


In [18]:
train_ts = data_to_np_tensor(train_data)
test_ts = data_to_np_tensor(test_data)

train_ts.shape, test_ts.shape

((28, 132, 5), (28, 24, 5))

In [19]:
train_ts

array([[[ 1.01617707e-02,  1.08784177e-02,  1.21508462e-02,
          8.62671952e-03, -5.79033902e-01],
        [ 5.20321443e-04,  2.99246130e-03,  3.76059475e-04,
         -4.04530744e-04,  8.68174461e-01],
        [ 8.08967988e-04, -4.36054851e-03, -4.62668440e-04,
          4.62507949e-04, -3.66862479e-01],
        ...,
        [ 3.52858151e-03,  3.93955430e-03,  2.98339930e-03,
          2.56440488e-03,  1.08821557e-01],
        [-4.68823254e-04,  0.00000000e+00,  1.08967751e-03,
          2.96945286e-03, -2.23905932e-01],
        [ 1.70028143e-03,  9.37097341e-04,  2.50058837e-03,
          2.93134783e-04,  5.41046204e-01]],

       [[-4.09643409e-03, -6.61322645e-03, -3.89654014e-03,
         -6.62251656e-03, -8.06811021e-01],
        [-6.67565745e-03, -1.47938942e-03, -6.81189721e-03,
         -6.73400673e-05,  1.40605428e+00],
        [-3.86939108e-03, -7.13852785e-03, -3.59907646e-03,
         -7.94666307e-03, -4.17516269e-01],
        ...,
        [ 2.85755108e-04,  1.0706638

In [20]:
model = TS2Vec(
    input_dims=train_ts.shape[2],
    device=2,
    output_dims=128
)

loss_log = model.fit(
    train_ts,
    verbose=False
)

In [21]:
train_repr = model.encode(train_ts)
test_repr = model.encode(test_ts)

train_repr.shape, test_repr.shape

((28, 132, 128), (28, 24, 128))

In [22]:
def stock_embeddigns_to_df(data_tensor: np.ndarray, stocks, dates) -> dict:    
    res = pd.DataFrame()
    for i, stock in enumerate(stocks):
        df = pd.DataFrame(data_tensor[i], index=dates)
        df.columns = ['emb_' + str(i) for i in range(len(df.columns))]
        df['Stock'] = stock
        res = pd.concat([res, df])

    return res.reset_index(drop=False, names='Datetime')

X_train = stock_embeddigns_to_df(train_repr, stocks=train_data['Open'].columns, dates=train_data['Open'].index)
X_test = stock_embeddigns_to_df(test_repr, stocks=test_data['Open'].columns, dates=test_data['Open'].index)
X_train.head()

,Datetime,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_119,emb_120,emb_121,emb_122,emb_123,emb_124,emb_125,emb_126,emb_127,Stock
0,2023-10-02 17:00:00,-0.385265,-0.123203,-0.091646,-0.204892,-0.034029,-0.085184,-0.201969,-0.228396,0.070437,...,-0.241715,0.060760,-0.279620,-0.041730,0.081007,-0.392990,-0.078081,-0.105887,-0.132299,AAPL
1,2023-10-02 18:00:00,-0.351957,-0.255687,-0.294856,-0.080170,-0.416529,0.022934,-0.527076,-0.070292,-0.141278,...,-0.019346,-0.037140,-0.114509,0.011569,-0.064772,0.018382,-0.261668,-0.132436,-0.192172,AAPL
2,2023-10-02 19:00:00,-0.128883,-0.273875,-0.303679,-0.131447,-0.006280,-0.187134,-0.054544,-0.113141,0.089145,...,-0.282725,0.068355,-0.410741,0.043855,0.088872,-0.251950,0.048974,-0.243150,-0.177219,AAPL
3,2023-10-02 20:00:00,-0.415582,-0.120054,-0.236820,-0.267131,-0.413341,-0.032708,-0.496844,-0.358852,-0.127075,...,0.030586,0.085363,-0.289977,-0.097256,-0.028461,-0.137592,-0.229315,-0.074737,-0.112864,AAPL
4,2023-10-02 21:00:00,-0.201604,-0.244662,-0.177347,-0.065351,-0.053743,-0.065106,-0.161435,-0.007014,0.081622,...,-0.253341,-0.007203,-0.173872,0.082190,0.129482,-0.247891,-0.111787,-0.310488,-0.250245,AAPL


In [23]:
X_emd = pd.concat([X_train, X_test])

In [24]:
X_test

,Datetime,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_119,emb_120,emb_121,emb_122,emb_123,emb_124,emb_125,emb_126,emb_127,Stock
0,2023-11-01 17:00:00,-0.504512,0.022980,-0.007553,-0.209028,-0.032553,-0.022900,-0.186269,-0.280322,-0.026118,...,-0.172211,0.015120,-0.051658,-0.089307,0.190460,-0.348224,-0.215933,0.022340,-0.045283,AAPL
1,2023-11-01 18:00:00,-0.179979,-0.175429,-0.283328,-0.052348,-0.312327,-0.067571,-0.278235,-0.054847,-0.146593,...,0.021192,-0.042385,-0.150826,0.122111,0.139642,-0.002298,-0.263469,-0.134470,-0.196267,AAPL
2,2023-11-01 19:00:00,-0.116160,-0.057031,-0.258859,-0.197705,-0.156470,-0.176774,-0.003421,-0.257454,-0.071712,...,-0.135529,0.012428,-0.398007,-0.014885,0.176926,-0.234081,-0.113674,-0.148711,-0.123626,AAPL
3,2023-11-01 20:00:00,-0.316153,0.067556,-0.158645,-0.278369,-0.286341,0.006026,-0.339078,-0.264047,-0.161310,...,0.036918,0.018340,-0.080790,-0.069316,0.159207,-0.248574,-0.330377,-0.108121,-0.086309,AAPL
4,2023-11-01 21:00:00,0.026167,-0.320324,-0.259313,0.010633,-0.144378,0.003073,-0.179600,0.212254,-0.181313,...,-0.159884,-0.210903,0.040812,0.137406,0.032105,-0.108681,-0.247408,-0.453238,-0.202339,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,2023-11-06 18:00:00,-0.423353,-0.074542,-0.189596,-0.150413,-0.355937,-0.029139,-0.519545,-0.300839,-0.121003,...,0.035223,0.014384,-0.138413,-0.028406,0.066079,-0.144026,-0.300620,0.007082,-0.167317,XOM
668,2023-11-06 19:00:00,-0.149557,-0.261403,-0.223415,-0.126792,-0.075474,-0.195349,-0.013328,-0.091172,0.055661,...,-0.179825,0.083767,-0.346477,0.092250,0.190184,-0.137902,-0.056755,-0.250590,-0.162810,XOM
669,2023-11-06 20:00:00,-0.300638,-0.122891,-0.294276,-0.225548,-0.312759,-0.104975,-0.277257,-0.284171,-0.131726,...,0.034140,0.058917,-0.282131,0.013066,0.127952,-0.130242,-0.140989,-0.067567,-0.113063,XOM
670,2023-11-06 21:00:00,-0.111392,-0.244962,-0.238885,-0.083403,-0.137289,-0.109434,-0.153449,0.051161,0.051914,...,-0.149552,0.034532,-0.229332,0.096231,0.164789,-0.114927,-0.174620,-0.308807,-0.202775,XOM


In [25]:
y = df_best.set_index('Datetime').groupby(
    ['Stock', pd.Grouper( freq='h')],
).agg({'Close': 'mean'}).reset_index()
y.head()

,Stock,Datetime,Close
0,AAPL,2023-01-30 17:00:00,144.833667
1,AAPL,2023-01-30 18:00:00,144.159833
2,AAPL,2023-01-30 19:00:00,143.951083
3,AAPL,2023-01-30 20:00:00,143.681667
4,AAPL,2023-01-30 21:00:00,143.388167


In [26]:
train_start, train_end = '2023-10-01', '2023-11-01'
test_start, test_end = '2023-11-01', '2023-11-07'

y_train = y[(y['Datetime'].dt.date >= pd.Timestamp(train_start).date()) & 
             (y['Datetime'].dt.date < pd.Timestamp(train_end).date())]

y_test = y[(y['Datetime'].dt.date >= pd.Timestamp(test_start).date()) & 
             (y['Datetime'].dt.date < pd.Timestamp(test_end).date())]

In [27]:
y_emd = pd.concat([y_train, y_test])

In [28]:
y_emd

,Stock,Datetime,Close
1011,AAPL,2023-10-02 17:00:00,173.344000
1012,AAPL,2023-10-02 18:00:00,173.639167
1013,AAPL,2023-10-02 19:00:00,173.219500
1014,AAPL,2023-10-02 20:00:00,173.224333
1015,AAPL,2023-10-02 21:00:00,173.018417
...,...,...,...
41851,XOM,2023-11-06 18:00:00,107.347500
41852,XOM,2023-11-06 19:00:00,107.014914
41853,XOM,2023-11-06 20:00:00,106.882000
41854,XOM,2023-11-06 21:00:00,106.277917


In [29]:
X_emd.to_csv('results/TS2Vec/x_emb.csv', index=False)
y_emd.to_csv('results/TS2Vec/y.csv', index=False)

In [30]:
y_name = 'Close'

df_train = pd.merge(X_train, y_train, on=['Stock', 'Datetime'])
df_test = pd.merge(X_test, y_test, on=['Stock', 'Datetime'])

X_train, y_train = df_train.drop(columns=[y_name, 'Datetime', 'Stock']), df_train[y_name]
X_test, y_test = df_test.drop(columns=[y_name, 'Datetime', 'Stock']), df_test[y_name]

In [31]:
model = XGBRegressor()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

MAE(y_test, y_pred)

88.38426280038455

# baseline

In [32]:
X_baseline = df_best.set_index('Datetime').groupby(
    ['Stock', pd.Grouper( freq='h')],
).agg({'Close': 'mean'}).reset_index()
X_baseline.head()

,Stock,Datetime,Close
0,AAPL,2023-01-30 17:00:00,144.833667
1,AAPL,2023-01-30 18:00:00,144.159833
2,AAPL,2023-01-30 19:00:00,143.951083
3,AAPL,2023-01-30 20:00:00,143.681667
4,AAPL,2023-01-30 21:00:00,143.388167


In [33]:
n_shifts = 18
for i in range(1, n_shifts + 1):
    X_baseline[f'shift_{i}'] = X_baseline.groupby(by=['Stock']).shift(i)['Close']

X_baseline = X_baseline.dropna()
X_baseline = X_baseline.drop(columns='Close')
X_baseline.head()

,Stock,Datetime,shift_1,shift_2,shift_3,shift_4,shift_5,shift_6,shift_7,shift_8,shift_9,shift_10,shift_11,shift_12,shift_13,shift_14,shift_15,shift_16,shift_17,shift_18
18,AAPL,2023-02-02 17:00:00,143.121271,142.186500,142.061917,142.751167,142.929167,143.026833,143.825508,143.472250,143.434917,143.335833,143.010000,142.988000,143.219153,143.388167,143.681667,143.951083,144.159833,144.833667
19,AAPL,2023-02-02 18:00:00,149.049167,143.121271,142.186500,142.061917,142.751167,142.929167,143.026833,143.825508,143.472250,143.434917,143.335833,143.010000,142.988000,143.219153,143.388167,143.681667,143.951083,144.159833
20,AAPL,2023-02-02 19:00:00,149.299833,149.049167,143.121271,142.186500,142.061917,142.751167,142.929167,143.026833,143.825508,143.472250,143.434917,143.335833,143.010000,142.988000,143.219153,143.388167,143.681667,143.951083
21,AAPL,2023-02-02 20:00:00,149.979417,149.299833,149.049167,143.121271,142.186500,142.061917,142.751167,142.929167,143.026833,143.825508,143.472250,143.434917,143.335833,143.010000,142.988000,143.219153,143.388167,143.681667
22,AAPL,2023-02-02 21:00:00,149.869333,149.979417,149.299833,149.049167,143.121271,142.186500,142.061917,142.751167,142.929167,143.026833,143.825508,143.472250,143.434917,143.335833,143.010000,142.988000,143.219153,143.388167


In [34]:
y = df_best.set_index('Datetime').groupby(
    ['Stock', pd.Grouper( freq='h')],
).agg({'Close': 'mean'}).reset_index()
y.head()

,Stock,Datetime,Close
0,AAPL,2023-01-30 17:00:00,144.833667
1,AAPL,2023-01-30 18:00:00,144.159833
2,AAPL,2023-01-30 19:00:00,143.951083
3,AAPL,2023-01-30 20:00:00,143.681667
4,AAPL,2023-01-30 21:00:00,143.388167


In [35]:
train_start, train_end = '2023-10-01', '2023-11-01'
test_start, test_end = '2023-11-01', '2023-11-07'

y_train = y[(y['Datetime'].dt.date >= pd.Timestamp(train_start).date()) & 
             (y['Datetime'].dt.date < pd.Timestamp(train_end).date())]

y_test = y[(y['Datetime'].dt.date >= pd.Timestamp(test_start).date()) & 
             (y['Datetime'].dt.date < pd.Timestamp(test_end).date())]

X_train = X_baseline[(X_baseline['Datetime'].dt.date >= pd.Timestamp(train_start).date()) & 
             (X_baseline['Datetime'].dt.date < pd.Timestamp(train_end).date())]

X_test = X_baseline[(X_baseline['Datetime'].dt.date >= pd.Timestamp(test_start).date()) & 
             (X_baseline['Datetime'].dt.date < pd.Timestamp(test_end).date())]

In [36]:
y_name = 'Close'

df_train = pd.merge(X_train, y_train, on=['Stock', 'Datetime'])
df_test = pd.merge(X_test, y_test, on=['Stock', 'Datetime'])

X_train, y_train = df_train.drop(columns=[y_name, 'Datetime', 'Stock']), df_train[y_name]
X_test, y_test = df_test.drop(columns=[y_name, 'Datetime', 'Stock']), df_test[y_name]

In [37]:
X_train

,shift_1,shift_2,shift_3,shift_4,shift_5,shift_6,shift_7,shift_8,shift_9,shift_10,shift_11,shift_12,shift_13,shift_14,shift_15,shift_16,shift_17,shift_18
0,171.311695,170.744583,171.005583,171.718583,172.111083,172.523333,170.999915,170.717667,170.963167,171.560167,170.608667,169.657333,170.279068,169.947333,169.615583,169.937500,170.682500,170.882167
1,173.344000,171.311695,170.744583,171.005583,171.718583,172.111083,172.523333,170.999915,170.717667,170.963167,171.560167,170.608667,169.657333,170.279068,169.947333,169.615583,169.937500,170.682500
2,173.639167,173.344000,171.311695,170.744583,171.005583,171.718583,172.111083,172.523333,170.999915,170.717667,170.963167,171.560167,170.608667,169.657333,170.279068,169.947333,169.615583,169.937500
3,173.219500,173.639167,173.344000,171.311695,170.744583,171.005583,171.718583,172.111083,172.523333,170.999915,170.717667,170.963167,171.560167,170.608667,169.657333,170.279068,169.947333,169.615583
4,173.224333,173.219500,173.639167,173.344000,171.311695,170.744583,171.005583,171.718583,172.111083,172.523333,170.999915,170.717667,170.963167,171.560167,170.608667,169.657333,170.279068,169.947333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3691,105.645167,105.760847,105.542583,105.651525,105.511695,105.422000,105.671333,105.205508,105.709667,105.430932,105.270000,105.476417,105.653333,107.394224,107.236983,106.939000,107.064375,107.374333
3692,105.780667,105.645167,105.760847,105.542583,105.651525,105.511695,105.422000,105.671333,105.205508,105.709667,105.430932,105.270000,105.476417,105.653333,107.394224,107.236983,106.939000,107.064375
3693,105.537583,105.780667,105.645167,105.760847,105.542583,105.651525,105.511695,105.422000,105.671333,105.205508,105.709667,105.430932,105.270000,105.476417,105.653333,107.394224,107.236983,106.939000
3694,105.716186,105.537583,105.780667,105.645167,105.760847,105.542583,105.651525,105.511695,105.422000,105.671333,105.205508,105.709667,105.430932,105.270000,105.476417,105.653333,107.394224,107.236983


In [38]:
y_train

0       173.344000
1       173.639167
2       173.219500
3       173.224333
4       173.018417
           ...    
3691    105.780667
3692    105.537583
3693    105.716186
3694    105.618500
3695    105.799746
Name: Close, Length: 3696, dtype: float64

In [39]:
model = XGBRegressor()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

MAE(y_test, y_pred)

2.1079210682349263

## Baseline + TS2Vec

In [40]:
shifts = X_train

In [41]:
shifts_test = X_test

In [42]:
def stock_embeddigns_to_df(data_tensor: np.ndarray, stocks, dates) -> dict:    
    res = pd.DataFrame()
    for i, stock in enumerate(stocks):
        df = pd.DataFrame(data_tensor[i], index=dates)
        df.columns = ['emb_' + str(i) for i in range(len(df.columns))]
        df['Stock'] = stock
        res = pd.concat([res, df])

    return res.reset_index(drop=False, names='Datetime')

X_train = stock_embeddigns_to_df(train_repr, stocks=train_data['Open'].columns, dates=train_data['Open'].index)
X_test = stock_embeddigns_to_df(test_repr, stocks=test_data['Open'].columns, dates=test_data['Open'].index)
X_train.head()

,Datetime,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_119,emb_120,emb_121,emb_122,emb_123,emb_124,emb_125,emb_126,emb_127,Stock
0,2023-10-02 17:00:00,-0.385265,-0.123203,-0.091646,-0.204892,-0.034029,-0.085184,-0.201969,-0.228396,0.070437,...,-0.241715,0.060760,-0.279620,-0.041730,0.081007,-0.392990,-0.078081,-0.105887,-0.132299,AAPL
1,2023-10-02 18:00:00,-0.351957,-0.255687,-0.294856,-0.080170,-0.416529,0.022934,-0.527076,-0.070292,-0.141278,...,-0.019346,-0.037140,-0.114509,0.011569,-0.064772,0.018382,-0.261668,-0.132436,-0.192172,AAPL
2,2023-10-02 19:00:00,-0.128883,-0.273875,-0.303679,-0.131447,-0.006280,-0.187134,-0.054544,-0.113141,0.089145,...,-0.282725,0.068355,-0.410741,0.043855,0.088872,-0.251950,0.048974,-0.243150,-0.177219,AAPL
3,2023-10-02 20:00:00,-0.415582,-0.120054,-0.236820,-0.267131,-0.413341,-0.032708,-0.496844,-0.358852,-0.127075,...,0.030586,0.085363,-0.289977,-0.097256,-0.028461,-0.137592,-0.229315,-0.074737,-0.112864,AAPL
4,2023-10-02 21:00:00,-0.201604,-0.244662,-0.177347,-0.065351,-0.053743,-0.065106,-0.161435,-0.007014,0.081622,...,-0.253341,-0.007203,-0.173872,0.082190,0.129482,-0.247891,-0.111787,-0.310488,-0.250245,AAPL


In [43]:
y = df_best.set_index('Datetime').groupby(
    ['Stock', pd.Grouper( freq='h')],
).agg({'Close': 'mean'}).reset_index()
y.head()

,Stock,Datetime,Close
0,AAPL,2023-01-30 17:00:00,144.833667
1,AAPL,2023-01-30 18:00:00,144.159833
2,AAPL,2023-01-30 19:00:00,143.951083
3,AAPL,2023-01-30 20:00:00,143.681667
4,AAPL,2023-01-30 21:00:00,143.388167


In [44]:
train_start, train_end = '2023-10-01', '2023-11-01'
test_start, test_end = '2023-11-01', '2023-11-07'

y_train = y[(y['Datetime'].dt.date >= pd.Timestamp(train_start).date()) & 
             (y['Datetime'].dt.date < pd.Timestamp(train_end).date())]

y_test = y[(y['Datetime'].dt.date >= pd.Timestamp(test_start).date()) & 
             (y['Datetime'].dt.date < pd.Timestamp(test_end).date())]

In [45]:
y_name = 'Close'

df_train = pd.merge(X_train, y_train, on=['Stock', 'Datetime'])
df_test = pd.merge(X_test, y_test, on=['Stock', 'Datetime'])

X_train, y_train = df_train.drop(columns=[y_name, 'Datetime', 'Stock']), df_train[y_name]
X_test, y_test = df_test.drop(columns=[y_name, 'Datetime', 'Stock']), df_test[y_name]

In [46]:
X_train = pd.concat([X_train, shifts], axis=1)
X_test = pd.concat([X_test, shifts_test], axis=1)

In [47]:
X_train

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,shift_9,shift_10,shift_11,shift_12,shift_13,shift_14,shift_15,shift_16,shift_17,shift_18
0,-0.385265,-0.123203,-0.091646,-0.204892,-0.034029,-0.085184,-0.201969,-0.228396,0.070437,-0.048320,...,170.963167,171.560167,170.608667,169.657333,170.279068,169.947333,169.615583,169.937500,170.682500,170.882167
1,-0.351957,-0.255687,-0.294856,-0.080170,-0.416529,0.022934,-0.527076,-0.070292,-0.141278,-0.055391,...,170.717667,170.963167,171.560167,170.608667,169.657333,170.279068,169.947333,169.615583,169.937500,170.682500
2,-0.128883,-0.273875,-0.303679,-0.131447,-0.006280,-0.187134,-0.054544,-0.113141,0.089145,-0.025335,...,170.999915,170.717667,170.963167,171.560167,170.608667,169.657333,170.279068,169.947333,169.615583,169.937500
3,-0.415582,-0.120054,-0.236820,-0.267131,-0.413341,-0.032708,-0.496844,-0.358852,-0.127075,-0.045087,...,172.523333,170.999915,170.717667,170.963167,171.560167,170.608667,169.657333,170.279068,169.947333,169.615583
4,-0.201604,-0.244662,-0.177347,-0.065351,-0.053743,-0.065106,-0.161435,-0.007014,0.081622,-0.042822,...,172.111083,172.523333,170.999915,170.717667,170.963167,171.560167,170.608667,169.657333,170.279068,169.947333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3691,-0.416403,0.025767,-0.170889,-0.091016,-0.316288,0.058204,-0.502588,-0.148468,-0.190860,0.025471,...,105.709667,105.430932,105.270000,105.476417,105.653333,107.394224,107.236983,106.939000,107.064375,107.374333
3692,0.183354,-0.277991,-0.376437,-0.046939,-0.041082,-0.277822,0.205418,0.120699,-0.132567,0.098866,...,105.205508,105.709667,105.430932,105.270000,105.476417,105.653333,107.394224,107.236983,106.939000,107.064375
3693,-0.430571,0.131885,-0.216386,-0.307566,-0.249061,-0.136346,-0.146051,-0.527754,-0.236913,0.108019,...,105.671333,105.205508,105.709667,105.430932,105.270000,105.476417,105.653333,107.394224,107.236983,106.939000
3694,-0.117654,-0.069224,-0.126597,-0.053445,-0.119163,-0.079515,-0.102096,0.054009,-0.116166,0.035370,...,105.422000,105.671333,105.205508,105.709667,105.430932,105.270000,105.476417,105.653333,107.394224,107.236983


In [48]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error as MAE
from xgboost import XGBRegressor

model = XGBRegressor()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

MAE(y_test, y_pred)

1.8273502935272692